Still need to figure out:
* Running mean

In [1]:
import numpy
import pandas
import xray; print(xray.__version__)

0.5.0


### Reading data

In [2]:
#infile = '/Users/damienirving/Downloads/Data/va_ERAInterim_500hPa_2006-030day-runmean_native.nc'
infile = '/Users/damienirving/Downloads/Data/sic_ERAInterim_surface_1993-1994-daily_native-shextropics30.nc'
dset = xray.open_dataset(infile)

In [3]:
dset

<xray.Dataset>
Dimensions:    (latitude: 81, longitude: 480, time: 730)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * time       (time) datetime64[ns] 1993-01-01T18:00:00 1993-01-02T18:00:00 ...
Data variables:
    sic        (time, latitude, longitude) float64 nan nan nan nan nan nan ...
Attributes:
    CDI: Climate Data Interface version 1.5.3 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.0
    history: Mon Jun 22 16:13:38 2015: cdo seldate,1993-01-01,1994-12-31 -sellonlatbox,0,359.9,-90,-30 sic_ERAInterim_surface_daily_native.nc sic_ERAInterim_surface_1993-1994-daily_native-shextropics30.nc
Thu Mar 19 13:05:10 2015: ncatted -O -a axis,time,c,c,T /mnt/meteo0/data/simmonds/dbirving/ERAInterim/data/sic_ERAInterim_surface_daily_native.nc
Thu Mar 19 11:42:24 2015: ncatted -O -a missing_value,sic,o,f,-32767. /mnt/meteo0/data/simmonds/dbirving/ERAInterim

In [4]:
darray = dset['sic']
darray

<xray.DataArray 'sic' (time: 730, latitude: 81, longitude: 480)>
[28382400 values with dtype=float64]
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 1993-01-01T18:00:00 1993-01-02T18:00:00 ...
Attributes:
    standard_name: sea_ice_fraction
    long_name: sea_ice_fraction
    units: (0 - 1)

In [5]:
# Get the order of the dimensions
darray.dims

(u'time', u'latitude', u'longitude')

In [6]:
# To swap order of axes
order = ['latitude', 'longitude', 'time']
dset['sic'].transpose(*order)

<xray.DataArray 'sic' (latitude: 81, longitude: 480, time: 730)>
array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  

In [7]:
darray['time'].attrs

OrderedDict([(u'standard_name', u'time')])

In [8]:
dset.dims.keys()

[u'latitude', u'longitude', u'time']

#### Missing values

[This page](http://xray.readthedocs.org/en/stable/io.html) suggests that xray recognises the `_FillValue` attribute.

However, once you've read the data in the `_FillValue` attribute disappears. When you write the output file there is no `_FillValue` attribute and any missing values appear as `NaNf` when you ncdump. This doesn't appear to be a problem for cdo or iris.

In [9]:
missing_data = darray.sel(longitude=3.75, latitude=-81).values
numpy.isnan(missing_data[0])

True

### Climatologies

You can group by `month`, `day`, `dayofyear`, `dayofweek` and whatever else belongs to the [`pandas.DatetimeIndex` class](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DatetimeIndex.html).

##### Monthly

In [10]:
mon_clim = darray.groupby('time.month').mean('time')

In [11]:
mon_clim

<xray.DataArray 'sic' (month: 12, latitude: 81, longitude: 480)>
array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,  

#### Daily

(Feb 28 is day 59. In a leap year, Feb 29 is day 60.)

In [12]:
day_clim = darray.groupby('time.dayofyear').mean('time')
day_clim

<xray.DataArray 'sic' (dayofyear: 365, latitude: 81, longitude: 480)>
array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   

#### Daily anomaly 

In [13]:
dummy_dset = xray.Dataset({'foo': ('time', numpy.random.randn(4000)),
                           'time': pandas.date_range('2000-01-01', periods=4000)})
clim = dummy_dset.groupby('time.dayofyear').mean()
anom = dummy_dset.groupby('time.dayofyear') - clim

In [14]:
dummy_dset

<xray.Dataset>
Dimensions:  (time: 4000)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 ...
Data variables:
    foo      (time) float64 -1.626 0.01791 -0.8639 -0.05431 -0.3551 0.3045 ...

In [15]:
clim

<xray.Dataset>
Dimensions:    (dayofyear: 366)
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    foo        (dayofyear) float64 -0.3503 -0.02708 0.4346 0.2729 0.09533 ...

In [16]:
anom

<xray.Dataset>
Dimensions:    (time: 4000)
Coordinates:
    dayofyear  (time) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * time       (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 ...
Data variables:
    foo        (time) float64 -1.275 0.04499 -1.298 -0.3272 -0.4505 0.01008 ...

### Selection

It's possible to select a single dimension value or a slice of values:

In [17]:
slice(-70, -50)

slice(-70, -50, None)

In [18]:
darray.sel(time=('1993-03-25'))

<xray.DataArray 'sic' (time: 1, latitude: 81, longitude: 480)>
array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]]])
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 1993-03-25T18:00:00
Attributes:
    standard_name: sea_ice_fraction
    long_name: sea_ice_fraction
    units: (0 - 1)

In [19]:
darray.sel(time=slice('1993-03-25', '1993-04-25'))

<xray.DataArray 'sic' (time: 32, latitude: 81, longitude: 480)>
array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   

For an arbitrary list of dimension values, you need to provide the exact date/time value (for single dates xray/pandas can guess what you meant).

In [20]:
darray['latitude'][0:20]

<xray.DataArray 'latitude' (latitude: 20)>
array([-90.  , -89.25, -88.5 , -87.75, -87.  , -86.25, -85.5 , -84.75,
       -84.  , -83.25, -82.5 , -81.75, -81.  , -80.25, -79.5 , -78.75,
       -78.  , -77.25, -76.5 , -75.75], dtype=float32)
Coordinates:
  * latitude  (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
Attributes:
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    axis: Y

In [21]:
darray.sel(latitude=[-89.25, -81])

<xray.DataArray 'sic' (time: 730, latitude: 2, longitude: 480)>
array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       ..., 
       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]]])
Coordinates:
  * latitude   (latitude) float32 -89.25 -81.0
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 1993-01-01T18:00:00 1993-01-02T18:00:00 ...
Attributes:
    standard_name: sea_ice_fraction
    long_name: sea_ice_fraction

In [67]:
darray.sel(time=[numpy.datetime64('1993-02-16'), numpy.datetime64('1994-08-23')],
           method='backfill')

# The method options are nearest, pad or backfill.
# When working with datetimes where all you really care about is the date part,
# use backfill.

<xray.DataArray 'sic' (time: 2, latitude: 81, longitude: 480)>
array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]]])
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...
  * time       (time) datetime64[ns] 1993-02-16T18:00:00 1994-08-23T18:00:00
Attributes:
    standard_name: sea_ice_fraction

#### Composites

In [33]:
time_list = ['1993-01-04T05:00:00.000000000+1100', '1993-01-10T05:00:00.000000000+1100',
             '1993-04-03T04:00:00.000000000+1000', '1993-04-11T04:00:00.000000000+1000',
             '1994-07-01T04:00:00.000000000+1000', '1994-07-05T04:00:00.000000000+1000',
             '1994-12-18T05:00:00.000000000+1100', '1994-12-30T05:00:00.000000000+1100']
dt_list = map(numpy.datetime64, time_list)
dt_list

[numpy.datetime64('1993-01-04T05:00:00.000000000+1100'),
 numpy.datetime64('1993-01-10T05:00:00.000000000+1100'),
 numpy.datetime64('1993-04-03T04:00:00.000000000+1000'),
 numpy.datetime64('1993-04-11T04:00:00.000000000+1000'),
 numpy.datetime64('1994-07-01T04:00:00.000000000+1000'),
 numpy.datetime64('1994-07-05T04:00:00.000000000+1000'),
 numpy.datetime64('1994-12-18T05:00:00.000000000+1100'),
 numpy.datetime64('1994-12-30T05:00:00.000000000+1100')]

In [73]:
subset = darray.sel(time=dt_list, latitude=[-66, -67], longitude=[72.3, 73], method='nearest')
subset

<xray.DataArray 'sic' (time: 8, latitude: 2, longitude: 2)>
array([[[ 0.11719286,  0.06582943],
        [ 0.43137045,  0.28674103]],

       [[ 0.07031572,  0.0387591 ],
        [ 0.33404544,  0.21749348]],

       [[ 0.57812095,  0.55846673],
        [ 0.79880366,  0.78964796]],

       [[ 0.72265881,  0.70785711],
        [ 0.88546229,  0.8827461 ]],

       [[ 0.91405857,  0.93294981],
        [ 0.94677491,  0.96182076]],

       [[ 0.95312285,  0.96066104],
        [ 0.9673447 ,  0.97750752]],

       [[ 0.56640166,  0.44824745],
        [ 0.81461249,  0.69616834]],

       [[ 0.34375048,  0.2781652 ],
        [ 0.67962706,  0.59859918]]])
Coordinates:
  * latitude   (latitude) float32 -66.0 -66.75
  * longitude  (longitude) float32 72.0 72.75
  * time       (time) datetime64[ns] 1993-01-03T18:00:00 1993-01-09T18:00:00 ...
Attributes:
    standard_name: sea_ice_fraction
    long_name: sea_ice_fraction
    units: (0 - 1)

In [74]:
seasonal_subset = subset.resample('Q-FEB', dim='time', how='mean', label='right')
seasonal_subset

<xray.DataArray 'sic' (time: 9, latitude: 2, longitude: 2)>
array([[[ 0.09375429,  0.05229426],
        [ 0.38270795,  0.25211725]],

       [[ 0.65038988,  0.63316192],
        [ 0.84213297,  0.83619703]],

       [[        nan,         nan],
        [        nan,         nan]],

       [[        nan,         nan],
        [        nan,         nan]],

       [[        nan,         nan],
        [        nan,         nan]],

       [[        nan,         nan],
        [        nan,         nan]],

       [[ 0.93359071,  0.94680543],
        [ 0.9570598 ,  0.96966414]],

       [[        nan,         nan],
        [        nan,         nan]],

       [[ 0.45507607,  0.36320632],
        [ 0.74711977,  0.64738376]]])
Coordinates:
  * latitude   (latitude) float32 -66.0 -66.75
  * longitude  (longitude) float32 72.0 72.75
  * time       (time) datetime64[ns] 1993-02-28 1993-05-31 1993-08-31 ...
Attributes:
    standard_name: sea_ice_fraction
    long_name: sea_ice_fraction
    units: (0 

In [75]:
seasonal_subset['time']

<xray.DataArray 'time' (time: 9)>
array(['1993-02-28T11:00:00.000000000+1100',
       '1993-05-31T10:00:00.000000000+1000',
       '1993-08-31T10:00:00.000000000+1000',
       '1993-11-30T11:00:00.000000000+1100',
       '1994-02-28T11:00:00.000000000+1100',
       '1994-05-31T10:00:00.000000000+1000',
       '1994-08-31T10:00:00.000000000+1000',
       '1994-11-30T11:00:00.000000000+1100',
       '1995-02-28T11:00:00.000000000+1100'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1993-02-28 1993-05-31 1993-08-31 ...

In [80]:
djf_indexes = pandas.to_datetime(seasonal_subset['time'].values).month == 2
djf_composite = seasonal_subset.loc[djf_indexes].mean(dim='time')
djf_composite

<xray.DataArray 'sic' (latitude: 2, longitude: 2)>
array([[ 0.27441518,  0.20775029],
       [ 0.56491386,  0.44975051]])
Coordinates:
  * latitude   (latitude) float32 -66.0 -66.75
  * longitude  (longitude) float32 72.0 72.75

In [82]:
annual_composite = darray.mean(dim='time')
annual_composite

<xray.DataArray 'sic' (latitude: 81, longitude: 480)>
array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.25 -88.5 -87.75 -87.0 -86.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 ...

### Maths / stats

#### Basic stats along a dimension

In [112]:
outdata = darray.mean(dim=['latitude', 'longitude'])

#### Running mean

I need to figure out how to apply the pandas [moving (rolling) statistics](http://pandas.pydata.org/pandas-docs/stable/computation.html#moving-rolling-statistics-moments) to an `xray.DataArray`. [This page](http://xray.readthedocs.org/en/stable/pandas.html) talks about how I might do that.

In [121]:
outdata.attrs = darray.attrs

In [122]:
outdata.attrs

OrderedDict([(u'standard_name', u'sea_ice_fraction'), (u'long_name', u'sea_ice_fraction'), (u'units', u'(0 - 1)')])

In [123]:
outdata.to_netcdf('test_xray.nc', format='NETCDF3_CLASSIC')

AttributeError: 'DataArray' object has no attribute 'to_netcdf'

#### From scratch

In [32]:
hello = numpy.arange(0, 730)
print hello.shape

(730,)


In [33]:
d = {}
d['time'] = data['time']
d['hello'] = (['time'], hello)

dset_out = xray.Dataset(d)

In [34]:
dset_out.attrs

OrderedDict()

In [101]:
dset_out['time'].attrs

OrderedDict([(u'standard_name', u'time')])

In [104]:
dset_out['hello'].attrs = {'standard_name': 'hello', 'units': 'smiles'}

In [105]:
dset_out

<xray.Dataset>
Dimensions:  (time: 730)
Coordinates:
  * time     (time) datetime64[ns] 1993-01-01T18:00:00 1993-01-02T18:00:00 ...
Data variables:
    hello    (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...

In [106]:
dset_out['hello'].attrs

OrderedDict([('units', 'smiles'), ('standard_name', 'hello')])

In [36]:
words = 'hello_world'
words.split(' ')

['hello_world']